<a href="https://colab.research.google.com/github/AvellinaLeong/NHM-Nannofossil-Segmentation-Project/blob/main/03_evaluation_val_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

# Set script location to own development space
MY_DEVELOPMENT_SPACE = '/content/drive/MyDrive/development/avellina/'
import os
os.chdir(MY_DEVELOPMENT_SPACE)
!pwd
!ls

Mounted at /content/drive/
/content/drive/MyDrive/development/avellina
detectron2  Detectron2_notebooks  Mask-RCNN  Morphometrics_notebooks  output


## Find Precision, Recall and Accuracy

In [2]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 8.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.4 MB/s

In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.3 ; cuda:  cu121
detectron2: 0.6


In [4]:
# Import Detectron2 and Detectron2 Logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [5]:
import os
import torch
import copy
import numpy as np
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader, transforms as T
from detectron2.structures import BoxMode
from detectron2.data import detection_utils as utils
from detectron2.engine import DefaultTrainer
import cv2
from detectron2.utils.visualizer import Visualizer, ColorMode

In [6]:
# Register datasets
register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/data/species_53/data/val/coco_val.json", "/content/drive/MyDrive/data/species_53/data/val")
register_coco_instances("my_dataset_test", {}, "/content/drive/MyDrive/data/species_53/data/test/coco_test.json", "/content/drive/MyDrive/data/species_53/data/test")

In [30]:
# Load metadata for visualization
val_metadata = MetadataCatalog.get("my_dataset_val")
test_metadata = MetadataCatalog.get("my_dataset_test")

# Get validation and test dataset dicts
val_dataset_dicts = DatasetCatalog.get("my_dataset_val")
test_dataset_dicts = DatasetCatalog.get("my_dataset_test")

[06/28 15:57:34 d2.data.datasets.coco]: Loaded 95 images in COCO format from /content/drive/MyDrive/data/species_53/data/val/coco_val.json
[06/28 15:57:34 d2.data.datasets.coco]: Loaded 96 images in COCO format from /content/drive/MyDrive/data/species_53/data/test/coco_test.json


## Define Augmentations and Load Configurations

In [7]:
# Define augmentations
from detectron2.data import transforms as T

augs = T.AugmentationList([
    T.RandomBrightness(0.9, 1.1),
    T.RandomFlip(prob=0.5),
    T.RandomRotation(angle=[-90, 90]),
    T.RandomSaturation(0.8, 1.2),
])

In [8]:
# Custom data mapper
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict["file_name"], format="BGR")

    # Get the annotations
    annos = dataset_dict.get("annotations", [])
    bbox_list = [BoxMode.convert(obj["bbox"], obj["bbox_mode"], BoxMode.XYXY_ABS) for obj in annos]
    bbox_list = np.array(bbox_list, dtype=np.float32)

    # Apply augmentations
    aug_input = T.AugInput(image, boxes=bbox_list)
    transforms = augs(aug_input)
    image = aug_input.image

    # Apply the same transforms to the annotations
    annos = [utils.transform_instance_annotations(obj, transforms, image.shape[:2]) for obj in annos]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))
    dataset_dict["instances"] = utils.filter_empty_instances(instances)

    return dataset_dict

In [9]:
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/data/species_53/Detectron2_Models/1"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.VAL = ("my_dataset_val",)
cfg.DATASETS.TEST = ("my_dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_1.pth")  # Make sure this path is correct
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000   #iterations -- can change
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 # Default is 512
cfg.TEST.EVAL_PERIOD = 500  # Evaluate every 500 iterations
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Number of classes

# Load the saved configuration from the YAML file
config_yaml_path = "/content/drive/MyDrive/data/species_53/Detectron2_Models/1/config_1.yaml"
cfg.merge_from_file(config_yaml_path)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "1", "model_final_1.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50

In [10]:
# Initialize the predictor
predictor = DefaultPredictor(cfg)

[06/28 14:58:33 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/data/species_53/Detectron2_Models/1/model_final_1.pth ...


## Precision and Recall on Validation

In [11]:
# Accuracy and Recall on validation
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")

results = inference_on_dataset(predictor.model, val_loader, evaluator)
print(results)

[06/28 14:58:44 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
[06/28 14:58:45 d2.data.datasets.coco]: Loaded 95 images in COCO format from /content/drive/MyDrive/data/species_53/data/val/coco_val.json
[06/28 14:58:45 d2.data.build]: Distribution of instances among all 1 categories:
|  category   | #instances   |
|:-----------:|:-------------|
| t_orionatus | 103          |
|             |              |
[06/28 14:58:45 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[06/28 14:58:45 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[06/28 14:58:45 d2.data.common]: Serializing 95 elements to byte tensors and concatenating them all ...
[06/28 14:58:45 d2.data.common]: Serialized dataset takes 0.03 MiB
[06/28 14:58:45 d2.evaluation.evaluator]: Start inference on 95 batches


/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[06/28 14:58:52 d2.evaluation.evaluator]: Inference done 11/95. Dataloading: 0.2374 s/iter. Inference: 0.0727 s/iter. Eval: 0.0003 s/iter. Total: 0.3104 s/iter. ETA=0:00:26
[06/28 14:58:57 d2.evaluation.evaluator]: Inference done 23/95. Dataloading: 0.3139 s/iter. Inference: 0.0731 s/iter. Eval: 0.0004 s/iter. Total: 0.3875 s/iter. ETA=0:00:27
[06/28 14:59:02 d2.evaluation.evaluator]: Inference done 35/95. Dataloading: 0.3226 s/iter. Inference: 0.0782 s/iter. Eval: 0.0004 s/iter. Total: 0.4013 s/iter. ETA=0:00:24
[06/28 14:59:07 d2.evaluation.evaluator]: Inference done 48/95. Dataloading: 0.3219 s/iter. Inference: 0.0835 s/iter. Eval: 0.0004 s/iter. Total: 0.4060 s/iter. ETA=0:00:19
[06/28 14:59:13 d2.evaluation.evaluator]: Inference done 60/95. Dataloading: 0.3294 s/iter. Inference: 0.0840 s/iter. Eval: 0.0004 s/iter. Total: 0.4139 s/iter. ETA=0:00:14
[06/28 14:59:18 d2.evaluation.evaluator]: Inference done 73/95. Dataloading: 0.3300 s/iter. Inference: 0.0836 s/iter. Eval: 0.0004 s/it

In [ ]:
# Precision and Recall on Validation Set Results

# Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.263
#  Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.412
#  Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.276
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.274
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.302
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.345
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.345
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.351
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000

In [12]:
# Save to csv (but only precision gets saved need to debug this later)
import csv
import os

# Save evaluation as a csv file in folder "Detectron2_Models"
evaluation_csv_file_path = "/content/drive/MyDrive/data/species_53/Detectron2_Models/evaluation/recall_precision_val.csv"
os.makedirs(os.path.dirname(evaluation_csv_file_path), exist_ok=True)

# Extract metrics
metrics = results['segm']  # 'segm' for segmentation mask, 'bbox' for bounding box

# Debug: Print the metrics dictionary to ensure it contains the expected keys
print("Metrics dictionary:", metrics)

#CSV header
header = [
    'Metric', 'AP', 'AP50', 'AP75', 'APs', 'APm', 'APl',
    'AR@1', 'AR@10', 'AR@100', 'ARs', 'ARm', 'ARl'
]

# Open CSV
with open(evaluation_csv_file_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    # Prepare data row
    data_row = [
        'Segmentation Mask',
        metrics.get('AP', ''), metrics.get('AP50', ''), metrics.get('AP75', ''), metrics.get('APs', ''), metrics.get('APm', ''), metrics.get('APl', ''),
        metrics.get('AR@1', ''), metrics.get('AR@10', ''), metrics.get('AR@100', ''), metrics.get('ARs', ''), metrics.get('ARm', ''), metrics.get('ARl', '')
    ]

    # Debug: Print the data row to verify its content before writing
    print("Data row:", data_row)

    writer.writerow(data_row)

Metrics dictionary: {'AP': 26.342023553406342, 'AP50': 41.22544619272258, 'AP75': 27.562465281545528, 'APs': nan, 'APm': 27.39668453378413, 'APl': 0.0}
Data row: ['Segmentation Mask', 26.342023553406342, 41.22544619272258, 27.562465281545528, nan, 27.39668453378413, 0.0, '', '', '', '', '', '']


## Precision and Recall on Test

In [15]:
# Accuracy and Recall on test
evaluator = COCOEvaluator("my_dataset_test", output_dir="./output")
test_loader = build_detection_test_loader(cfg, "my_dataset_test")
results = inference_on_dataset(predictor.model, test_loader, evaluator)
print(results)

[06/28 15:15:27 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
[06/28 15:15:29 d2.data.datasets.coco]: Loaded 96 images in COCO format from /content/drive/MyDrive/data/species_53/data/test/coco_test.json
[06/28 15:15:29 d2.data.build]: Distribution of instances among all 1 categories:
|  category   | #instances   |
|:-----------:|:-------------|
| t_orionatus | 108          |
|             |              |
[06/28 15:15:29 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[06/28 15:15:29 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[06/28 15:15:29 d2.data.common]: Serializing 96 elements to byte tensors and concatenating them all ...
[06/28 15:15:29 d2.data.common]: Serialized dataset takes 0.03 MiB
[06/28 15:15:29 d2.evaluation.evaluator]: Start inference on 96 batches


/content/drive/MyDrive/development/avellina/detectron2/detectron2/layers/wrappers.py:142: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  x = F.conv2d(


[06/28 15:15:39 d2.evaluation.evaluator]: Inference done 23/96. Dataloading: 0.3048 s/iter. Inference: 0.1121 s/iter. Eval: 0.0005 s/iter. Total: 0.4175 s/iter. ETA=0:00:30
[06/28 15:15:44 d2.evaluation.evaluator]: Inference done 37/96. Dataloading: 0.3171 s/iter. Inference: 0.0956 s/iter. Eval: 0.0004 s/iter. Total: 0.4133 s/iter. ETA=0:00:24
[06/28 15:15:50 d2.evaluation.evaluator]: Inference done 51/96. Dataloading: 0.3210 s/iter. Inference: 0.0897 s/iter. Eval: 0.0004 s/iter. Total: 0.4113 s/iter. ETA=0:00:18
[06/28 15:15:56 d2.evaluation.evaluator]: Inference done 65/96. Dataloading: 0.3211 s/iter. Inference: 0.0862 s/iter. Eval: 0.0004 s/iter. Total: 0.4079 s/iter. ETA=0:00:12
[06/28 15:16:01 d2.evaluation.evaluator]: Inference done 77/96. Dataloading: 0.3275 s/iter. Inference: 0.0844 s/iter. Eval: 0.0004 s/iter. Total: 0.4124 s/iter. ETA=0:00:07
[06/28 15:16:06 d2.evaluation.evaluator]: Inference done 91/96. Dataloading: 0.3246 s/iter. Inference: 0.0835 s/iter. Eval: 0.0004 s/it

In [ ]:
# Precision and Recall on Test Set Results
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
#  Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.281
#  Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.225
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.222
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.251
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.282
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.282
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.288
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000

In [16]:
# Save evaluation as a CSV file
evaluation_csv_file_path = "/content/drive/MyDrive/data/species_53/Detectron2_Models/evaluation/recall_precision_test.csv"
os.makedirs(os.path.dirname(evaluation_csv_file_path), exist_ok=True)

# Extract relevant metrics from results['segm']
metrics = results['segm']  # Assuming 'segm' contains segmentation mask metrics

#CSV header
header = [
    'Metric', 'AP', 'AP50', 'AP75', 'APs', 'APm', 'APl',
    'AR@1', 'AR@10', 'AR@100', 'ARs', 'ARm', 'ARl'
]

# Open CSV
with open(evaluation_csv_file_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    # Prepare data row
    data_row = [
        'Segmentation Mask',
        metrics.get('AP', ''), metrics.get('AP50', ''), metrics.get('AP75', ''), metrics.get('APs', ''), metrics.get('APm', ''), metrics.get('APl', ''),
        metrics.get('AR@1', ''), metrics.get('AR@10', ''), metrics.get('AR@100', ''), metrics.get('ARs', ''), metrics.get('ARm', ''), metrics.get('ARl', '')
    ]

    # Debug: Print the data row to verify its content before writing
    print("Data row:", data_row)

    writer.writerow(data_row)

Data row: ['Segmentation Mask', 20.11010074019501, 28.067482710285717, 22.535429716817312, nan, 22.152352982125365, 0.0, '', '', '', '', '', '']


## Accuracy on Validation and Test

In [21]:
pip install shapely rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 67.0 MB/s eta 0:00:00


In [31]:
import numpy as np
from shapely.geometry import Polygon
import rasterio.features
from detectron2.data import DatasetCatalog, MetadataCatalog

def accuracy(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask * groundtruth_mask)
    union = np.sum(pred_mask) + np.sum(groundtruth_mask) - intersect
    xor = np.sum(groundtruth_mask == pred_mask)
    acc = np.mean(xor / (union + xor - intersect))
    return round(acc, 3)

def convert_polygons_to_mask(polygons, height, width):
    mask = np.zeros((height, width), dtype=np.uint8)

    try:
        for polygon in polygons:
            # Create a Shapely polygon object from the list of points
            poly = Polygon(polygon)

            # Rasterize the polygon into the mask
            if poly.is_valid:  # Check if the polygon is valid
                try:
                    # Use rasterio.features to rasterize the polygon
                    mask_shape = [(poly, 1)]
                    mask = rasterio.features.geometry_mask(mask_shape, out_shape=(height, width))

                    # Ensure mask is boolean (optional)
                    mask = mask.astype(np.uint8)  # Convert to uint8 (0 or 1)
                except Exception as e:
                    print(f"Error rasterizing polygon: {e}")
                    continue
            else:
                print("Invalid polygon detected, skipping...")
    except Exception as e:
        print(f"Error converting polygons to mask: {e}")

    return mask

def compute_accuracy_on_dataset(dataset_name):
    dataset_dicts = DatasetCatalog.get(dataset_name)
    total_accuracy = 0
    count = 0

    for dataset_dict in dataset_dicts:
        image = utils.read_image(dataset_dict["file_name"], format="BGR")
        outputs = predictor(image)
        pred_masks = outputs["instances"].pred_masks.cpu().numpy()

        if "annotations" in dataset_dict:
            annos = dataset_dict["annotations"]
            gt_masks = []
            for anno in annos:
                if 'segmentation' in anno:
                    masks = convert_polygons_to_mask(anno["segmentation"], image.shape[0], image.shape[1])
                    gt_masks.append(masks)
            gt_masks = np.array(gt_masks)

            if len(gt_masks) > 0 and len(pred_masks) > 0:
                gt_mask_combined = np.max(gt_masks, axis=0)
                pred_mask_combined = np.max(pred_masks, axis=0)
                acc = accuracy(gt_mask_combined, pred_mask_combined)
                total_accuracy += acc
                count += 1

    return total_accuracy / count if count > 0 else 0

# Calculate accuracy on both test and val datasets
val_accuracy = compute_accuracy_on_dataset("my_dataset_val")
test_accuracy = compute_accuracy_on_dataset("my_dataset_test")

print(f"Validation Accuracy: {val_accuracy}")
print(f"Test Accuracy: {test_accuracy}")


[06/28 15:57:36 d2.data.datasets.coco]: Loaded 95 images in COCO format from /content/drive/MyDrive/data/species_53/data/val/coco_val.json
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to mask: 'int' object is not iterable
Error converting polygons to 

Validation Accuracy: 0.9666799999999998
Test Accuracy: 0.966127659574468

## IOU on Validation and Test



*   Each IOU
*   Average IOU



In [35]:
# Get segmentation mask predictions for val and test datasets
val_predictions = []
test_predictions = []

for d in val_dataset_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    val_predictions.append(outputs["instances"].to("cpu").get("pred_masks"))

for d in test_dataset_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    test_predictions.append(outputs["instances"].to("cpu").get("pred_masks"))

In [36]:
def calculate_iou(predicted_masks, gt_masks):
    """
    Calculate IoU between predicted masks and ground truth masks.
    Args:
        predicted_masks (list): List of predicted segmentation masks.
        gt_masks (list): List of ground truth segmentation masks.
    Returns:
        list: List of IoU values.
    """
    ious = []
    for pred_mask, gt_mask in zip(predicted_masks, gt_masks):
        iou = compute_iou(pred_mask, gt_mask)
        ious.append(iou)

    return ious

def compute_iou(pred_mask, gt_mask):
    """
    Compute IoU between predicted mask and ground truth mask.
    Args:
        pred_mask (numpy.ndarray): Predicted segmentation mask (binary).
        gt_mask (numpy.ndarray): Ground truth segmentation mask (binary).
    Returns:
        float: IoU value.
    """
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    iou = intersection / union if union > 0 else 0.0
    return iou

# Example usage to calculate IoU for validation and test sets
val_gt_masks = [cv2.imread(d["file_name"])[:, :, 0] for d in val_dataset_dicts]  # Assuming ground truth masks are stored as images
test_gt_masks = [cv2.imread(d["file_name"])[:, :, 0] for d in test_dataset_dicts]  # Assuming ground truth masks are stored as images

val_ious = calculate_iou(val_predictions, val_gt_masks)
test_ious = calculate_iou(test_predictions, test_gt_masks)

# Print or process the IoU values as needed
print("IoU for validation set:")
print(val_ious)

print("IoU for test set:")
print(test_ious)

IoU for validation set:
[0.0, 0.0, tensor(0.0381), tensor(0.0298), tensor(0.0354), tensor(0.0474), 0.0, tensor(0.0381), 0.0, 0.0, 0.0, tensor(0.0177), tensor(0.0085), tensor(0.0054), tensor(0.0199), 0.0, tensor(0.0324), tensor(0.0301), tensor(0.0451), 0.0, 0.0, 0.0, tensor(0.0213), 0.0, 0.0, 0.0, 0.0, tensor(0.0432), tensor(0.0309), tensor(0.0202), tensor(0.0309), 0.0, tensor(0.0105), 0.0, tensor(0.0380), tensor(0.0223), tensor(0.0254), 0.0, tensor(0.0443), tensor(0.0469), 0.0, tensor(0.0492), tensor(0.0193), 0.0, tensor(0.0253), 0.0, 0.0, tensor(0.0385), tensor(0.0276), tensor(0.0415), 0.0, 0.0, 0.0, 0.0, tensor(0.0153), tensor(0.0313), tensor(0.0319), 0.0, 0.0, 0.0, tensor(0.0437), tensor(0.0271), tensor(0.0316), tensor(0.0340), 0.0, 0.0, tensor(0.0137), tensor(0.0318), tensor(0.0358), 0.0, 0.0, 0.0, 0.0, 0.0, tensor(0.0196), tensor(0.0290), tensor(0.0229), tensor(0.0339), 0.0, tensor(0.0365), tensor(0.0231), 0.0, tensor(0.0296), tensor(0.0295), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, tensor(0

In [37]:
import numpy as np

def calculate_iou(predicted_masks, gt_masks):
    """
    Calculate IoU between predicted masks and ground truth masks.
    Args:
        predicted_masks (list): List of predicted segmentation masks.
        gt_masks (list): List of ground truth segmentation masks.
    Returns:
        float: Average IoU value.
    """
    ious = []
    for pred_mask, gt_mask in zip(predicted_masks, gt_masks):
        iou = compute_iou(pred_mask, gt_mask)
        ious.append(iou)

    # Calculate average IoU
    avg_iou = np.mean(ious)

    return avg_iou

def compute_iou(pred_mask, gt_mask):
    """
    Compute IoU between predicted mask and ground truth mask.
    Args:
        pred_mask (numpy.ndarray): Predicted segmentation mask (binary).
        gt_mask (numpy.ndarray): Ground truth segmentation mask (binary).
    Returns:
        float: IoU value.
    """
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    iou = intersection / union if union > 0 else 0.0
    return iou

# Example usage to calculate IoU for validation and test sets
val_gt_masks = [cv2.imread(d["file_name"])[:, :, 0] for d in val_dataset_dicts]  # Assuming ground truth masks are stored as images
test_gt_masks = [cv2.imread(d["file_name"])[:, :, 0] for d in test_dataset_dicts]  # Assuming ground truth masks are stored as images

# Calculate average IoU for validation set
avg_val_iou = calculate_iou(val_predictions, val_gt_masks)

# Calculate average IoU for test set
avg_test_iou = calculate_iou(test_predictions, test_gt_masks)

# Print or process the average IoU values as needed
print(f"Average IoU for validation set: {avg_val_iou:.4f}")
print(f"Average IoU for test set: {avg_test_iou:.4f}")


Average IoU for validation set: 0.0157
Average IoU for test set: 0.0143


## Boundary F1 Score for Validation and Test


CAN'T FIGURE OUT -- code not working


In [39]:
# Function to get segmentation mask predictions for a dataset
def get_segmentation_predictions(dataset_dicts, predictor):
    predictions = []
    for d in dataset_dicts:
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        predictions.append(outputs["instances"].to("cpu").pred_masks)
    return predictions

# Get segmentation mask predictions for val and test datasets
val_predictions = get_segmentation_predictions(val_dataset_dicts, predictor)
test_predictions = get_segmentation_predictions(test_dataset_dicts, predictor)

In [ ]:
import numpy as np
from skimage.measure import find_contours
from sklearn.metrics import f1_score


def extract_boundary(mask):
    """
    Extract boundary from a binary mask using find_contours.
    Args:
        mask (numpy.ndarray): Binary segmentation mask.
    Returns:
        numpy.ndarray: Binary mask of the boundary.
    """
    contours = find_contours(mask, 0.5)
    boundary = np.zeros_like(mask, dtype=np.uint8)

    for contour in contours:
        contour = np.round(contour).astype(np.int32)
        boundary[contour[:, 0], contour[:, 1]] = 1

    return boundary

def calculate_boundary_f1(predicted_masks, gt_masks):
    """
    Calculate Boundary F1 Score between predicted masks and ground truth masks.
    Args:
        predicted_masks (list): List of predicted segmentation masks.
        gt_masks (list): List of ground truth segmentation masks.
    Returns:
        float: Boundary F1 Score.
    """
    boundary_f1_scores = []

    for pred_mask, gt_mask in zip(predicted_masks, gt_masks):
        # Example check for empty masks (adjust as needed)
        if np.count_nonzero(pred_mask) < 4 or np.count_nonzero(gt_mask) < 4:
            continue  # Skip empty or very small masks

        pred_boundary = extract_boundary(pred_mask)
        gt_boundary = extract_boundary(gt_mask)

        # Calculate F1 score based on boundaries
        boundary_f1 = f1_score(gt_boundary.flatten(), pred_boundary.flatten())
        boundary_f1_scores.append(boundary_f1)

    # Calculate average Boundary F1 Score
    avg_boundary_f1 = np.mean(boundary_f1_scores)

    return avg_boundary_f1

# Example usage to calculate Boundary F1 Score for validation and test sets
val_predictions = get_segmentation_predictions(val_dataset_dicts, predictor)
test_predictions = get_segmentation_predictions(test_dataset_dicts, predictor)

val_gt_masks = [cv2.imread(d["file_name"], cv2.IMREAD_GRAYSCALE) for d in val_dataset_dicts]
test_gt_masks = [cv2.imread(d["file_name"], cv2.IMREAD_GRAYSCALE) for d in test_dataset_dicts]

# Calculate Boundary F1 Score for validation set
avg_boundary_f1_val = calculate_boundary_f1(val_predictions, val_gt_masks)

# Calculate Boundary F1 Score for test set
avg_boundary_f1_test = calculate_boundary_f1(test_predictions, test_gt_masks)

# Print or process the Boundary F1 Scores as needed
print("Boundary F1 Score for validation set:", avg_boundary_f1_val)
print("Boundary F1 Score for test set:", avg_boundary_f1_test)
